In [1]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
import io
from google.colab import files
uploaded = files.upload()
test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))

Saving test.csv to test.csv


In [163]:
#geojson of boroughs
boroughs = gpd.read_file('http://data.beta.nyc//dataset/68c0332f-c3bb-4a78-a0c1-32af515892d6/resource/7c164faa-4458-4ff2-9ef0-09db00b509ef/download/42c737fd496f4d6683bba25fb0e86e1dnycboroughboundaries.geojson')
#centroids of manhattan
ny_y = boroughs.loc[boroughs.borough=='Manhattan','geometry'].centroid.apply(lambda p: p.y).reset_index(drop=True)[0]
ny_x = boroughs.loc[boroughs.borough=='Manhattan','geometry'].centroid.apply(lambda p: p.x).reset_index(drop=True)[0]
#folium map
map = folium.Map(location = [ny_y,ny_x], zoom_start = 11)
#add boroughs to map
colors = {boroughs.borough.unique()[0]:'blue',boroughs.borough.unique()[1]:'red',boroughs.borough.unique()[2]:'light blue', boroughs.borough.unique()[3]:'green',boroughs.borough.unique()[4]:'orange'}
#have to duplicate because colors are overridden in a loop
folium.GeoJson(boroughs.loc[boroughs.borough == 'Queens',:].geometry, 
            style_function=lambda feature: {
      'fillColor': colors['Queens'],
      'color' : colors['Queens'],
      'weight' : 1,
      'fillOpacity' : 0.15,
      }
            ).add_to(map)
folium.GeoJson(boroughs.loc[boroughs.borough == 'Staten Island',:].geometry, 
            style_function=lambda feature: {
      'fillColor': colors['Staten Island'],
      'color' : colors['Staten Island'],
      'weight' : 1,
      'fillOpacity' : 0.15,
      }
            ).add_to(map)
folium.GeoJson(boroughs.loc[boroughs.borough == 'Brooklyn',:].geometry, 
            style_function=lambda feature: {
      'fillColor': colors['Brooklyn'],
      'color' : colors['Brooklyn'],
      'weight' : 1,
      'fillOpacity' : 0.15,
      }
            ).add_to(map)
folium.GeoJson(boroughs.loc[boroughs.borough == 'Manhattan',:].geometry, 
            style_function=lambda feature: {
      'fillColor': colors['Manhattan'],
      'color' : colors['Manhattan'],
      'weight' : 1,
      'fillOpacity' : 0.15,
      }
            ).add_to(map)
folium.GeoJson(boroughs.loc[boroughs.borough == 'Bronx',:].geometry, 
            style_function=lambda feature: {
      'fillColor': colors['Bronx'],
      'color' : colors['Bronx'],
      'weight' : 1,
      'fillOpacity' : 0.15,
      }
            ).add_to(map)


In [0]:
#convert test to a geodataframe
test['geometry'] = test.apply(lambda x: Point(float(x.pickup_longitude), float(x.pickup_latitude)), axis=1)
test['geometry_drop'] = test.apply(lambda x: Point(float(x.dropoff_longitude), float(x.dropoff_latitude)), axis=1)
test_geo = gpd.GeoDataFrame(test, crs = map.crs, geometry = test.geometry)
#adding time of day
def time_of_day(num):
    if num >= 5 and num < 9:
      return 'Early Morning'
    elif num >= 9 and num < 12:
      return 'Morning'
    elif num >= 12 and num < 17:
      return 'Afternoon'
    elif num >= 17 and num < 20:
      return 'Evening'
    else:
      return 'Night'
test_geo['time_of_day'] = pd.to_numeric(test_geo.pickup_datetime.str[11:13]).apply(time_of_day)

In [0]:
#adding journeys to map, color-coded by time of day
for row in test_geo[0:1000].iterrows():
    row_values = row[1]
    pickup_location = [row_values['pickup_latitude'], row_values['pickup_longitude']]
    dropoff_location = [row_values['dropoff_latitude'], row_values['dropoff_longitude']]
    popup = row_values['pickup_datetime']
    if row_values['time_of_day'] == 'Early Morning':
      color = 'yellow'
    elif row_values['time_of_day'] == 'Morning':
      color = 'orange'
    elif row_values['time_of_day'] == 'Afternoon':
      color = 'red'
    elif row_values['time_of_day'] == 'Evening':
      color = 'blue'
    else:
      color = 'black'
    folium.PolyLine([pickup_location, dropoff_location], popup=popup, opacity=.5, color=color, weight=1).add_to(map)

In [165]:
map